In [18]:
import requests
from bs4 import BeautifulSoup

### Every linkedin job search page carries 25 jobs

In [71]:
def get_job_links(keyword, start_page, pages):
    def custom_selector(tag):
	    return tag.name == "a" and tag.has_attr("href") and keyword in tag.get('href')
    
    job_links = []
    position = start_page
    position = position*25 + 1
    try:
        for page in range(pages):
            url = f"https://www.linkedin.com/jobs/search/?currentJobId=3693533935&distance=25&geoId=102454443&keywords=data%20analyst&origin=JOB_SEARCH_PAGE_KEYWORD_HISTORY&refresh=true&start={position}"
            response = requests.get(url)
            soup = BeautifulSoup(response.text,'html.parser')
            tags = soup.find_all(custom_selector)
            for tag in tags:
                job_links.append(tag.get('href'))
            position = position*25 + 1
    except Exception as e:
        print(f'Error at page {page})
        print(e)
    finally:
        return job_links

In [73]:
links = get_job_links('data-analyst', start_page=0, pages=100)
print(len(links))

138
